In [ ]:
# import all of the packages we're going to use
import matplotlib.pyplot as plt
import numpy as np
import copy
import EXOSIMS.MissionSim
import astropy.units as u
import scipy
from matplotlib import ticker

In [ ]:
# set up plotting
%matplotlib widget

# 1. Building and Interacting with Simulation Objects

In order to create a mission simulation, we need an input specification.  Let's define one with all default values, except for a real input star catalog:

In [ ]:
specs = {
 "modules": {
 "PlanetPopulation": " ",
 "StarCatalog": "HWOMissionStars",
 "OpticalSystem": " ",
 "ZodiacalLight": " ",
 "BackgroundSources": " ",
 "PlanetPhysicalModel": " ",
 "Observatory": " ",
 "TimeKeeping": " ",
 "PostProcessing": " ",
 "Completeness": " ",
 "TargetList": " ",
 "SimulatedUniverse": " ",
 "SurveySimulation": " ",
 "SurveyEnsemble": " "
 }
}

We now create a `MissionSim` object, which will automatically build and pre-compute all the quantities we need to simulate a mission:

In [ ]:
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))

Note that a *lot* of defaults have been set. Let's take a look at some of the default values being used:

In [ ]:
print(f"Pupil Diameter: {sim.OpticalSystem.pupilDiam}")
print(f"Single-observation integration time limit: {sim.OpticalSystem.intCutoff}")
print(f"Observing mode central wavelength: {sim.OpticalSystem.observingModes[0]['lam']}") 
print(f"Observing mode IWA/OWA: {sim.OpticalSystem.observingModes[0]['IWA']}/{sim.OpticalSystem.observingModes[0]['OWA']}")

We can also see what kind of contrast and throughput we're expecting.  These are both functions of wavelength and angular separation - we'll check at the central wavelength and inner working angle:

In [ ]:
print("Starlight Suppression system contrast @0.1 arcsec: "
      f"{sim.OpticalSystem.starlightSuppressionSystems[0]['core_contrast'](500*u.nm, 0.1*u.arcsec)[0] :.2e}")
print("Starlight Suppression system core throughput @0.1 arcsec: "
      f"{sim.OpticalSystem.starlightSuppressionSystems[0]['core_thruput'](500*u.nm, 0.1*u.arcsec)[0]}")

We can also get some information about our science instrument:

In [ ]:
print("Science Instrument QE @500 nm: "
      f"{sim.OpticalSystem.scienceInstruments[0]['QE'](500*u.nm)[0]}")
print(f"Science Instrument pixel scale: {sim.OpticalSystem.scienceInstruments[0]['pixelScale']}")
print(f"Science Instrument dark current: {sim.OpticalSystem.scienceInstruments[0]['idark']}")
print(f"Science Instrument read noise (e): {sim.OpticalSystem.scienceInstruments[0]['sread']}")

Let's also take a look at our target list:

In [ ]:
print(f"Number of targets: {sim.TargetList.nStars}")
print(f"Min/Max Target V mag: {sim.TargetList.Vmag.min()}/{sim.TargetList.Vmag.max()}")
print(f"Min/Max Target distance: {sim.TargetList.dist.min()}/{sim.TargetList.dist.max()}")

If you read the HWO Mission star list documentation, you'll notice that it includes 163 targets.  Why are only 134 included here?  Let's ask the code to explain:

In [ ]:
specs["explainFiltering"] = True
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))

~30 of the targets have bright companions within 10 arcsec, and are removed by default.  We can prevent this, if desired:

In [ ]:
specs["filterBinaries"] = False
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))

In [ ]:
print(f"Number of targets: {sim.TargetList.nStars}")
print(f"Min/Max Target V mag: {sim.TargetList.Vmag.min()}/{sim.TargetList.Vmag.max()}")
print(f"Min/Max Target distance: {sim.TargetList.dist.min()}/{sim.TargetList.dist.max()}")

# 2. Target Achievable $\Delta$mag Values

We can now see how all of these translate to achievable $\Delta\textrm{mag}$ values for our target list.  We'll look at the maximum integration time value and the saturation value (effectively infinite integration time):

In [ ]:
TL = sim.TargetList
inds = np.argsort(TL.Vmag)
plt.figure()
plt.scatter(TL.Vmag[inds], TL.intCutoff_dMag[inds], label="Max. Int Time $\\Delta$mag")
plt.scatter(TL.Vmag[inds], TL.saturation_dMag[inds], label="Saturation $\\Delta$mag")
plt.legend()

That's odd.  I thought we had $10^{-10}$ contrast. Why are we saturating at only a bit above 23 and not closer to 25 $\Delta\textrm{mag}$? 

The reason is that our integration time model assumes a noise floor set by the residual starlight leaking through the coronagraph. This is attenuated by an assumed post-processing gain. There is also a stability factor that models overall PSF stability. Let's see what that these values default to:

In [ ]:
print(f"Post-Processing gain: {TL.PostProcessing._outspec['ppFact']}")
print(f"Stability Factor: {TL.OpticalSystem.stabilityFact}")

Well, that's the problem right there.  We're not attenuating the residual speckle at all.  Let's fix that and see how it changes things.  We'll assume that we can beat down the speckle residual (via some form of post-processing) by a factor of 10:

In [ ]:
specs["ppFact"] = 0.1
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))

In [ ]:
TL = sim.TargetList
inds = np.argsort(TL.Vmag)
plt.figure()
plt.scatter(TL.Vmag[inds], TL.intCutoff_dMag[inds], label="Max. Int Time $\\Delta$mag")
plt.scatter(TL.Vmag[inds], TL.saturation_dMag[inds], label="Saturation $\\Delta$mag")
plt.legend()
plt.xlabel("V magnitude")
plt.ylabel("$\\Delta$mag");

Much better! 

# 3. Planet Populations

Now let's take a look at what kind of synthetic planets we're creating:

In [ ]:
print(f"Assumed occurrence rate: {sim.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim.SimulatedUniverse.nPlans}")

Q: Why is the number of planets not exactly equal to $\eta N_\textrm{stars}$? 

A: Because we treat $\eta$ as the rate parameter of a Poisson random variable.  

We're not producing a lot of planets, so for visualization purposes, let's increase the occurrence rate:

In [ ]:
specs["eta"] = 3
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))
print(f"Assumed occurrence rate: {sim.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim.SimulatedUniverse.nPlans}")

Let's take a look at some of the planets' physical and orbital attributes:

In [ ]:
fig, ax = plt.subplots()
pts = ax.scatter(sim.SimulatedUniverse.a, sim.SimulatedUniverse.Mp, s = sim.SimulatedUniverse.Rp, c = sim.SimulatedUniverse.Rp)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim.SimulatedUniverse.a.unit}]");
plt.colorbar(pts,label=f"Planet Radius [{sim.SimulatedUniverse.Rp.unit}]");

That seems like a really odd-looking planet population.  Because we haven't selected a specific planet population to model, planets were generated using log-normal distributions for mass and semi-major axis, and the planet radius is decoupled from the planet mass.  This is obviously non-physical (although occasionally useful for various testing purposes) - let's dial in a real planet population. We'll use the population defined by the SAG13 final report:

In [ ]:
specs["modules"]["PlanetPopulation"] = "SAG13"
specs["modules"]["SimulatedUniverse"] = "SAG13Universe"
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))
print(f"Assumed occurrence rate: {sim.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim.SimulatedUniverse.nPlans}")

Note that our previously set $\eta$ input was ignored and overwritten by this particular family of modules. 

In [ ]:
fig, ax = plt.subplots()
pts = ax.scatter(sim.SimulatedUniverse.a, sim.SimulatedUniverse.Mp, s = sim.SimulatedUniverse.Rp, c = sim.SimulatedUniverse.Rp)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim.SimulatedUniverse.a.unit}]");
plt.colorbar(pts,label=f"Planet Radius [{sim.SimulatedUniverse.Rp.unit}]");

We are now generating planets with self-consistent masses and radii, and generating significantly more Earth-mass objects than Jovian-mass objects. However, the original SAG13 population is only defined for relatively short-period planets.  We can see the exact range of semi-major axes we're generating:

In [ ]:
print(f"Semi-major axis range: {sim.PlanetPopulation.arange}")

Let's extrapolate a bit:

In [ ]:
specs["arange"] = [0.09084645, 30]
specs["smaknee"] = 10
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))
print(f"Assumed occurrence rate: {sim.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim.SimulatedUniverse.nPlans}")

In [ ]:
fig, ax = plt.subplots()
pts = ax.scatter(sim.SimulatedUniverse.a, sim.SimulatedUniverse.Mp, s = sim.SimulatedUniverse.Rp, c = sim.SimulatedUniverse.Rp)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim.SimulatedUniverse.a.unit}]");
plt.colorbar(pts,label=f"Planet Radius [{sim.SimulatedUniverse.Rp.unit}]");

Note that even though we permitted the semi-major axis range to go all the way to 30 AU, we're not generating planets out there.  That's because this implementation includes an exponential drop-off past the separation set by the ``smaknee`` parameter, which we set to 10 AU.

# 4. Completeness

At this point, we have a (somewhat) reasonable-looking set of synthetic planets, a target list, and a workable optical system.  The last element we're missing is the ability compute completeness.  So far, we have been utilizing the prototype completeness modules, which doesn't actually compute real completeness values (in the interest of execution time):

In [ ]:
sim.TargetList.int_comp

Let's replace this with a real completeness calculation, and also limit ourselves to a somewhat restricted field of view:

In [ ]:
specs["modules"]["Completeness"] = "BrownCompleteness" # Monte Carlo-based calculation
specs["FoV"] = 2.0 # arcseconds
sim = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs))

We can now look at what kind of completeness values we can achieve for our target list.  We'll consider the case of integrating to a nominal $\Delta$mag (defaulting to 25) for each target, and the case of integrating for an effectively infinite amount of time:

In [ ]:
TL = sim.TargetList
inds = np.argsort(TL.dist)
plt.figure()
plt.scatter(TL.dist[inds], TL.int_comp[inds], label="Nominal integraton time completeness")
plt.scatter(TL.dist[inds], TL.saturation_comp[inds], label="Saturation completeness")
plt.legend()
plt.xlabel(f"Distance {TL.dist.unit}")
plt.ylabel("Completeness");

This gives us a sense of how the completeness behaves as a function of integration time for two data points.  We can also look in more detail for a particular target.  Let's pick a somewhat bright star beyond 5 parsecs:

In [ ]:
sInd = np.where((TL.Vmag <= 7) & (TL.dist >= 5*u.pc))[0][0]
print(f"We'll be focusing on {TL.Name[sInd]}.")
print(f"This target has a V mag of {TL.Vmag[sInd]} and is {TL.dist[sInd]} from Earth.")
print(f"This target has a saturation dMag of {TL.saturation_dMag[sInd] :.2f}, "
      f"and requires {TL.int_tmin[sInd].to(u.h) :.2f} of integration to get to a dMag "
      f"of {TL.int_dMag[sInd]}")

Let's take a look at how instrumental constraints interact with the planet population for this target:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cs = ax.contourf(
    TL.Completeness.xnew,
    TL.Completeness.ynew,
    TL.Completeness.Cpdf,
    locator=ticker.LogLocator(),
)
ax.set_xlabel("Separation (AU)")
ax.set_ylabel("$\Delta$mag")
cbar = fig.colorbar(cs)
cbar.ax.set_title("log$_{10}(f_{\Delta\\mathrm{mag},s})$")
projIWA = np.tan(TL.default_mode["IWA"])*TL.dist[sInd]
projOWA = np.tan(TL.default_mode["OWA"])*TL.dist[sInd]
ax.plot([projIWA.to(u.AU).value] * 2, [10, 50], "--", label="IWA")
ax.plot([projOWA.to(u.AU).value] * 2, [10, 50], "--", label="OWA")
ax.plot([projIWA.to(u.AU).value,projOWA.to(u.AU).value], [TL.int_dMag[sInd]]*2, label="Nominal integration dMag")
ax.plot([projIWA.to(u.AU).value,projOWA.to(u.AU).value], [TL.saturation_dMag[sInd]]*2, label="Nominal integration dMag")
plt.legend()
ax.set_xlim([0,30])
ax.set_ylim([10, 50]);

The heat map represents the 2D joint probability density function of this population's projected separation and delta magnitude.  The lines represent instrumental limits. Note that the $\Delta$mag curves are straight lines because the default starlight-suppression system values use the same contrast and throughput values for all separations between the inner and outer working angles.

We'll now compute integration times for a range of $\Delta$mag values and also confirm that the saturation $\Delta$mag is correct:

In [ ]:
dMags1 = np.linspace(17, TL.saturation_dMag[sInd] + 0.1, 100) # target delta mag values
sInds = np.array([sInd] * len(dMags1))
fZ = [TL.ZodiacalLight.fZ0.value] * len(dMags1) * TL.ZodiacalLight.fZ0.unit # local zodiacal light
fEZ = [TL.ZodiacalLight.fEZ0.value] * len(dMags1) * TL.ZodiacalLight.fEZ0.unit # exo-zodiacal light
mode = TL.OpticalSystem.observingModes[0] # observing mode
WAs = [TL.int_WA[sInd].value] * len(sInds) * TL.int_WA.unit # use coronagraph parameters at this nominal separation
intTimes1 = TL.OpticalSystem.calc_intTime(TL, sInds, fZ, fEZ, dMags1, WAs, mode)

plt.figure()
plt.semilogy(dMags1, intTimes1)
plt.xlabel("$\\Delta$mag")
plt.ylabel(f"Integration Time {intTimes1.unit}");

Note that all integration time values above the saturation $\Delta$mag are NaN (infeasible):

In [ ]:
print(intTimes1[dMags1 > TL.saturation_dMag[sInd]])

We can now compute completeness as a function of integration time:

In [ ]:
intTimes1 = intTimes1[dMags1 < TL.saturation_dMag[sInd]]
dMags1 = dMags1[dMags1 < TL.saturation_dMag[sInd]]
projIWA = np.tan(mode["IWA"]) * TL.dist[sInd] # projected IWA
projOWA = np.tan(mode["OWA"]) * TL.dist[sInd] # projected OWA
comp1 = TL.Completeness.comp_calc(projIWA.to(u.AU).value, projOWA.to(u.AU).value, dMags1)

plt.figure()
plt.semilogx(intTimes1, comp1)
plt.xlabel(f"Integration Time [{intTimes1.unit}]");
plt.ylabel("Completeness");

Note that the first few completeness values are exactly zero.  These correspond to points where the limiting $\Delta$mag line lies entirely below the planet population heat map.

We can also invert the integration time calculation to compute the achievable $\Delta$mag as a function of integration time:

In [ ]:
sInds = np.array([sInd] * len(dMags1))
fZ = [TL.ZodiacalLight.fZ0.value] * len(dMags1) * TL.ZodiacalLight.fZ0.unit # local zodiacal light
fEZ = [TL.ZodiacalLight.fEZ0.value] * len(dMags1) * TL.ZodiacalLight.fEZ0.unit # exo-zodiacal light
WAs = [TL.int_WA[sInd].value] * len(sInds) * TL.int_WA.unit # use coronagraph parameters at this nominal separation
dMags2 = TL.OpticalSystem.calc_dMag_per_intTime(intTimes1, TL, sInds, fZ, fEZ, WAs, mode)
print(f"Maximum difference: {np.max(np.abs(dMags1 - dMags2))}")

Finally, we have the ability to evaluate the rate of change of completeness as a function of integration time (formally, the derivative $\frac{\mathrm{d}c}{\mathrm{d}t}$):

In [ ]:
dcdt1 = TL.Completeness.dcomp_dt(
            intTimes1,
            TL,
            sInds,
            TL.ZodiacalLight.fZ0,
            TL.ZodiacalLight.fEZ0,
            TL.int_WA[sInd],
            mode,
        ).to("1/d")

plt.figure()
plt.semilogx(intTimes1, dcdt1)
plt.xlabel(f"Integration Time [{intTimes1.unit}]");
plt.ylabel("$\\frac{\\mathrm{d}c}{\\mathrm{d}t}$");

Let's take a look at how much we've added to our input specification:

In [ ]:
specs

# 5.  Exercise: Choose another star and re-create some or all of these calculations

Have fun!